In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib as plt
from shapely.validation import make_valid


## Philippine River Dataset 

This dataset contains all types of waterways located in the Philippines. For this assessment we will only be utilizing waterway specifically `river` and `stream`. 

In [3]:
river_df = gpd.read_file("hotosm_phl_waterways_lines_shp.shp")
river_df


,name,name_en,waterway,covered,width,depth,layer,blockage,tunnel,natural,water,source,name_fil,osm_id,osm_type,geometry
0,Cagayan River,None,river,None,None,None,None,None,None,None,None,landsat,None,909866165,ways_line,"LINESTRING (121.34927 15.99296, 121.34949 15.9..."
1,None,None,river,None,None,None,None,None,None,None,None,None,None,245941601,ways_line,"LINESTRING (124.87041 11.16783, 124.87038 11.1..."
2,None,None,stream,None,None,None,None,None,None,None,None,None,None,502889972,ways_line,"LINESTRING (124.97525 10.88445, 124.97532 10.8..."
3,None,None,stream,None,None,None,-1,None,culvert,None,None,None,None,1184195783,ways_line,"LINESTRING (125.51376 7.05871, 125.51389 7.0586)"
4,None,None,stream,None,None,None,None,None,None,None,None,None,None,1317786873,ways_line,"LINESTRING (123.39114 12.27959, 123.39137 12.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121214,Angono River,None,river,None,None,None,None,None,None,None,None,None,None,15246527,relations,"MULTILINESTRING ((121.17283 14.54152, 121.1724..."
121215,Pamoantogbo River,None,river,None,None,None,None,None,None,None,None,None,None,13036436,relations,"MULTILINESTRING ((122.8461 7.53302, 122.8462 7..."
121216,Odiongan River,None,river,None,None,None,None,None,None,None,None,None,None,12720902,relations,"LINESTRING (125.19814 8.82519, 125.19764 8.825..."
121217,Mandulog River,None,river,None,None,None,None,None,None,None,None,None,None,12715038,relations,"LINESTRING (124.24519 8.25548, 124.24604 8.255..."


In [4]:
river_final = river_df[river_df["waterway"].isin(["river","stream"])]
river_final = river_final[["name", "waterway", "geometry"]]

river_final

,name,waterway,geometry
0,Cagayan River,river,"LINESTRING (121.34927 15.99296, 121.34949 15.9..."
1,None,river,"LINESTRING (124.87041 11.16783, 124.87038 11.1..."
2,None,stream,"LINESTRING (124.97525 10.88445, 124.97532 10.8..."
3,None,stream,"LINESTRING (125.51376 7.05871, 125.51389 7.0586)"
4,None,stream,"LINESTRING (123.39114 12.27959, 123.39137 12.2..."
...,...,...,...
121214,Angono River,river,"MULTILINESTRING ((121.17283 14.54152, 121.1724..."
121215,Pamoantogbo River,river,"MULTILINESTRING ((122.8461 7.53302, 122.8462 7..."
121216,Odiongan River,river,"LINESTRING (125.19814 8.82519, 125.19764 8.825..."
121217,Mandulog River,river,"LINESTRING (124.24519 8.25548, 124.24604 8.255..."


### Exploratory Data Analysis 

In [5]:
# This would just be a sanity check to ensure that all data are in Lat/lon degrees
if river_final.crs is None or river_final.crs.to_epsg() != 4326:
    river_final = river_final.to_crs(4326)

In [6]:
river_final.isnull().sum()

name        88416
waterway        0
geometry        0
dtype: int64

In [7]:
river_final.duplicated().sum()


np.int64(6)

In [8]:
river_final[river_final.duplicated()]


,name,waterway,geometry
121111,Lubogan River,river,"LINESTRING (125.42837 7.05983, 125.42845 7.059..."
121114,Dalanas River,river,"LINESTRING (122.16361 11.37186, 122.16361 11.3..."
121115,Litoban River,river,"LINESTRING (122.2797 7.68224, 122.27956 7.6822..."
121170,Bitanagan River,river,"LINESTRING (126.19383 7.07723, 126.19354 7.077..."
121212,Anayan River,river,"LINESTRING (120.85559 17.83885, 120.85547 17.8..."
121216,Odiongan River,river,"LINESTRING (125.19814 8.82519, 125.19764 8.825..."


In [9]:
river_cleaned_df = river_final.dropna()
river_cleaned_df = river_cleaned_df.drop_duplicates()

In [10]:
river_cleaned_df

,name,waterway,geometry
0,Cagayan River,river,"LINESTRING (121.34927 15.99296, 121.34949 15.9..."
7,Lopoy Creek,stream,"LINESTRING (125.41293 7.06458, 125.41301 7.064..."
23,Tuganay River,river,"LINESTRING (125.51876 7.56096, 125.51912 7.560..."
47,Matiao River,river,"LINESTRING (126.07515 7.23193, 126.07495 7.232..."
56,Calumpang River,river,"LINESTRING (121.07517 13.76744, 121.07496 13.7..."
...,...,...,...
121213,Bugsuk River,river,"LINESTRING (117.29818 8.29036, 117.29876 8.291..."
121214,Angono River,river,"MULTILINESTRING ((121.17283 14.54152, 121.1724..."
121215,Pamoantogbo River,river,"MULTILINESTRING ((122.8461 7.53302, 122.8462 7..."
121217,Mandulog River,river,"LINESTRING (124.24519 8.25548, 124.24604 8.255..."


## OCHA Dataset

In [11]:
# Load province (ADM2) and city/municipality (ADM3) shapefiles
adm2 = gpd.read_file("phl_adm_psa_namria_20231106_shp/phl_admbnda_adm2_psa_namria_20231106.shp").to_crs(4326)
adm3 = gpd.read_file("phl_adm_psa_namria_20231106_shp/phl_admbnda_adm3_psa_namria_20231106.shp").to_crs(4326)

print(adm2.columns[:6])   # usually includes ADM2_EN, ADM1_EN
print(adm3.columns[:6])   # usually includes ADM3_EN, ADM2_EN



Index(['ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'ADM0_EN',
       'ADM0_PCODE'],
      dtype='object')
Index(['ADM3_EN', 'ADM3_PCODE', 'ADM2_EN', 'ADM2_PCODE', 'ADM1_EN',
       'ADM1_PCODE'],
      dtype='object')


In [12]:
# Start with your cleaned HOT-OSM rivers
rivers = river_cleaned_df.copy()

# First join: province (ADM2)
rivers = gpd.sjoin(rivers, adm2[["ADM1_EN","ADM2_EN","geometry"]],
                   how="left", predicate="intersects")

# Second join: city/municipality (ADM3)
rivers = gpd.sjoin(rivers, adm3[["ADM3_EN","ADM2_EN","geometry"]],
                   how="left", predicate="intersects",
                   lsuffix="prov", rsuffix="mun")


In [13]:
river_city = rivers[[
    "name", "waterway",
    "ADM1_EN", "ADM2_EN_prov", "ADM3_EN",
    "geometry"
]].reset_index(drop=True)


In [14]:
river_city 

,name,waterway,ADM1_EN,ADM2_EN_prov,ADM3_EN,geometry
0,Cagayan River,river,Region II (Cagayan Valley),Nueva Vizcaya,Alfonso Castaneda,"LINESTRING (121.34927 15.99296, 121.34949 15.9..."
1,Lopoy Creek,stream,Region XI (Davao Region),Davao del Sur,Davao City,"LINESTRING (125.41293 7.06458, 125.41301 7.064..."
2,Tuganay River,river,Region XI (Davao Region),Davao del Sur,Davao City,"LINESTRING (125.51876 7.56096, 125.51912 7.560..."
3,Tuganay River,river,Region XI (Davao Region),Davao del Sur,Santo Tomas,"LINESTRING (125.51876 7.56096, 125.51912 7.560..."
4,Tuganay River,river,Region XI (Davao Region),Davao del Norte,Davao City,"LINESTRING (125.51876 7.56096, 125.51912 7.560..."
...,...,...,...,...,...,...
13818,Angono River,river,Region IV-A (Calabarzon),Rizal,Angono,"MULTILINESTRING ((121.17283 14.54152, 121.1724..."
13819,Pamoantogbo River,river,Region IX (Zamboanga Peninsula),Zamboanga Sibugay,Payao,"MULTILINESTRING ((122.8461 7.53302, 122.8462 7..."
13820,Mandulog River,river,Region X (Northern Mindanao),Lanao del Norte,Iligan City,"LINESTRING (124.24519 8.25548, 124.24604 8.255..."
13821,Gamay River,river,Region VIII (Eastern Visayas),Northern Samar,Gamay,"LINESTRING (125.23851 12.41278, 125.23868 12.4..."


Notice that our rows increased from <b> 9,528 </b> to <b> 13,823 </b>. We treat this increase not as an error but because it means that some rivers flows across multiple provinces/city. 

## River Catchment Characteristics Dataset

This dataset includes <b> catchment polygons, stream networks, rainfall/runoff attributes, and mean flow statistics </b> which allows us to identify <b> hydrological basics feeding each river, </b> and estimate water discharge. By using this data we are able to get an estimate on its possible usage for run-of-river projects. 

In [ ]:
# ============================================================
# Hydropower screening for HOT-OSM rivers & streams (PHL)
# - One row per original feature (no geometry explosion)
# - Catchment attributes (n128) + proximity to n128 stream net
# - Type-aware thresholds and 0–100 hydro score
# ============================================================


# -------------------- CONFIG --------------------
TOL_M_NEAR = 200   # consider a line "on_n128" if <= this distance (m)
TOL_M_FBK  = 500   # fallback tolerance to nearest catchment if rep-point misses (m)

# -------------------- INPUTS --------------------
# 1) HOT-OSM rivers/streams you've already cleaned and enriched with ADM fields
rivers = river_city[river_city["waterway"].isin(["river", "stream"])].copy()
if rivers.crs is None or rivers.crs.to_epsg() != 4326:
    rivers = rivers.to_crs(4326)
expected_rows = len(rivers)

# 2) n128 catchments & stream network
catch_river  = gpd.read_file(
    "characteristics/Philippines_GIS_catchments_n128/philippines_gis_catchments_n128.shp"
).to_crs(4326)

catch_stream = gpd.read_file(
    "characteristics/Philippines_GIS_stream_network_n128/philippines_gis_stream_network_n128.shp"
).to_crs(4326)

# -------------------- A) ONE CATCHMENT PER LINE (1:1 guaranteed) --------------------
# Stable key + representative point
rivers = rivers.reset_index(drop=True)
rivers["row_id"] = rivers.index
rivers["rep_pt"] = rivers.geometry.representative_point()
pts = gpd.GeoDataFrame(rivers.drop(columns="geometry"), geometry="rep_pt", crs=4326)

# Keep only known catchment fields
catch_keep = [c for c in [
    "New_Name","Catch_area","Catch_elev","Catch_slp","Catch_hyp","Drainage_D","Catch_rel","geometry"
] if c in catch_river.columns]

# Dissolve by New_Name to remove internal overlaps/multiparts (prevents 1→many)
catch_base = catch_river[catch_keep].copy()
if "New_Name" in catch_base.columns:
    catch_base = catch_base.dissolve(by="New_Name", as_index=False)

# Primary: point-in-polygon (rep-point within catchment)
join1 = gpd.sjoin(pts, catch_base, how="left", predicate="within")

# If sjoin brings index_right (match id), use polygon area to keep dominant match per row_id
if "index_right" in join1.columns:
    poly_area = catch_base.to_crs(3857).area
    area_tbl  = (catch_base.reset_index()
                          .rename(columns={"index":"index_right"})
                          .assign(poly_area=poly_area.values)[["index_right","poly_area"]])
    join1 = join1.merge(area_tbl, on="index_right", how="left")
    join1 = (join1.sort_values(["row_id","poly_area"], ascending=[True, False])
                   .drop_duplicates("row_id"))
    join1 = join1.drop(columns=[c for c in ["index_right","poly_area"] if c in join1.columns])
else:
    join1 = join1.sort_values("row_id").drop_duplicates("row_id")

# Normalize name
if "New_Name" in join1.columns:
    join1 = join1.rename(columns={"New_Name":"Catchment"})

# Fallback: nearest polygon for any misses (align by row_id)
miss_mask = join1["Catchment"].isna() if "Catchment" in join1.columns else pd.Series(False, index=join1.index)
if miss_mask.any():
    miss_pts = rivers.loc[miss_mask, ["row_id","rep_pt"]].copy()
    miss_pts = gpd.GeoDataFrame(miss_pts.set_geometry("rep_pt"), geometry="rep_pt", crs=4326).to_crs(3857)

    polys_m  = catch_base.to_crs(3857)
    fb = gpd.sjoin_nearest(miss_pts, polys_m, how="left", distance_col="d_near")
    fb = fb[fb["d_near"] <= TOL_M_FBK].to_crs(4326)

    if "New_Name" in fb.columns:
        fb = fb.rename(columns={"New_Name":"Catchment"})

    assign_cols = [c for c in ["Catchment","Catch_area","Catch_elev","Catch_slp","Catch_hyp","Drainage_D","Catch_rel"] if c in fb.columns]
    fb_small = fb[["row_id"] + assign_cols].drop_duplicates("row_id")

    join1 = join1.merge(fb_small.add_suffix("_fb"), left_on="row_id", right_on="row_id_fb", how="left")
    for col in assign_cols:
        join1[col] = join1[col].fillna(join1[f"{col}_fb"])
    drop_helpers = ["row_id_fb"] + [f"{c}_fb" for c in assign_cols]
    join1 = join1.drop(columns=[c for c in drop_helpers if c in join1.columns])

# Restore original line geometry
if "rep_pt" in join1.columns:
    join1 = join1.drop(columns=["rep_pt"])
rivers_catch = gpd.GeoDataFrame(join1, geometry=rivers.geometry, crs=4326)

# Enforce 1:1
rivers_catch = rivers_catch.sort_values("row_id").drop_duplicates("row_id")
assert len(rivers_catch) == expected_rows, f"[Catchment step] Expected {expected_rows}, got {len(rivers_catch)}"

# -------------------- B) n128 STREAM ATTRIBUTES & DISTANCE --------------------
# Try to keep an order/strahler column + an id if present
order_candidates = [c for c in catch_stream.columns if c.lower() in {"order","strahler","streamord"}]
id_candidates    = [c for c in catch_stream.columns if c.lower() in {"stream_id","river_id","rid","fid","id"}]
keep_cols = (order_candidates[:1] + id_candidates[:1])

cs_use = catch_stream[keep_cols + ["geometry"]] if keep_cols else catch_stream[["geometry"]].copy()

riv_m = rivers_catch.to_crs(3857)
cs_m  = cs_use.to_crs(3857)
near  = gpd.sjoin_nearest(riv_m, cs_m, how="left", distance_col="dist_m").to_crs(4326)

# Keep only the single nearest per row_id
near = near.sort_values(["row_id","dist_m"]).drop_duplicates("row_id")

# Flag if aligned to the modeled network
near["on_n128"] = near["dist_m"].notna() & (near["dist_m"] <= TOL_M_NEAR)
if keep_cols:
    near.loc[~near["on_n128"], keep_cols] = np.nan

assert len(near) == expected_rows, f"[n128 step] Expected {expected_rows}, got {len(near)}"

# -------------------- C) TYPE-AWARE HYDRO SCREEN + SCORE --------------------
for c in ["Catch_area","Catch_elev","Catch_slp","Catch_rel"]:
    if c in near.columns:
        near[c] = pd.to_numeric(near[c], errors="coerce")

def hydro_rule_typeaware(row):
    if row["waterway"] == "stream":
        area_min, slope_min, relief_min, elev_min = 50, 8, 80, 120
    else:
        area_min, slope_min, relief_min, elev_min = 100, 10, 100, 150
    return (
        (row.get("Catch_area", np.nan) >= area_min) and
        (row.get("Catch_slp",  np.nan) >= slope_min) and
        (row.get("Catch_rel",  np.nan) >= relief_min) and
        (row.get("Catch_elev", np.nan) >= elev_min)
    )

def hydro_score(row):
    a = np.clip((row.get("Catch_area", 0)-50)/(300-50),   0, 1)
    s = np.clip((row.get("Catch_slp",  0)-5) /(20-5),     0, 1)
    r = np.clip((row.get("Catch_rel",  0)-50)/(300-50),   0, 1)
    e = np.clip((row.get("Catch_elev", 0)-100)/(600-100), 0, 1)
    return round(100*(0.35*a + 0.25*r + 0.25*s + 0.15*e), 1)

near["hydro_usable"] = near.apply(lambda x: "Yes" if hydro_rule_typeaware(x) else "No", axis=1)
near["hydro_score"]  = near.apply(hydro_score, axis=1)

# -------------------- D) FINAL TIDY ----------------------------------------
final_cols = [
    "row_id","name","waterway","ADM1_EN","ADM2_EN_prov","ADM3_EN",
    "Catchment","Catch_area","Catch_elev","Catch_slp","Catch_hyp","Drainage_D","Catch_rel",
    *keep_cols, "dist_m","on_n128",
    "hydro_usable","hydro_score","geometry"
]
final_cols = [c for c in final_cols if c in near.columns]
final_hydro = near[final_cols].reset_index(drop=True)


Row count (should equal HOT-OSM filtered): 13823 vs 13823
By type: {'river': 10795, 'stream': 3028}
Usable by type:
 waterway  hydro_usable
river     No              6664
          Yes             4131
stream    No              2046
          Yes              982
Name: count, dtype: int64


In [16]:
final_hydro

,row_id,name,waterway,ADM1_EN,ADM2_EN_prov,ADM3_EN,Catchment,Catch_area,Catch_elev,Catch_slp,Catch_hyp,Drainage_D,Catch_rel,streamord,dist_m,on_n128,hydro_usable,hydro_score,geometry
0,0,Cagayan River,river,Region II (Cagayan Valley),Nueva Vizcaya,Alfonso Castaneda,Cagayan,27684.068,514.605,16.619,0.179,0.832,2871.037,3.0,21.601520,True,Yes,91.8,"LINESTRING (121.34927 15.99296, 121.34949 15.9..."
1,1,Lopoy Creek,stream,Region XI (Davao Region),Davao del Sur,Davao City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9092.042719,False,No,NaN,"LINESTRING (125.41293 7.06458, 125.41301 7.064..."
2,2,Tuganay River,river,Region XI (Davao Region),Davao del Sur,Davao City,Tagum,3167.920,213.310,15.334,0.140,0.951,1528.282,NaN,3590.409163,False,Yes,80.6,"LINESTRING (125.51876 7.56097, 125.51912 7.560..."
3,3,Tuganay River,river,Region XI (Davao Region),Davao del Sur,Santo Tomas,Tagum,3167.920,213.310,15.334,0.140,0.951,1528.282,NaN,3590.409163,False,Yes,80.6,"LINESTRING (125.51876 7.56097, 125.51912 7.560..."
4,4,Tuganay River,river,Region XI (Davao Region),Davao del Norte,Davao City,Tagum,3167.920,213.310,15.334,0.140,0.951,1528.282,NaN,3590.409163,False,Yes,80.6,"LINESTRING (125.51876 7.56097, 125.51912 7.560..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13818,13818,Angono River,river,Region IV-A (Calabarzon),Rizal,Angono,Marikina_Pasig,3960.976,155.505,7.225,0.072,0.976,2154.866,2.0,0.000000,True,No,65.4,"MULTILINESTRING ((121.17283 14.54152, 121.1724..."
13819,13819,Pamoantogbo River,river,Region IX (Zamboanga Peninsula),Zamboanga Sibugay,Payao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12154.562002,False,No,NaN,"MULTILINESTRING ((122.8461 7.53302, 122.8462 7..."
13820,13820,Mandulog River,river,Region X (Northern Mindanao),Lanao del Norte,Iligan City,Mandulog,776.031,633.527,16.620,0.452,0.813,1400.139,5.0,0.000000,True,Yes,94.4,"LINESTRING (124.24519 8.25548, 124.24604 8.255..."
13821,13821,Gamay River,river,Region VIII (Eastern Visayas),Northern Samar,Gamay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3999.619763,False,No,NaN,"LINESTRING (125.23851 12.41278, 125.23868 12.4..."


## Philippine National Hydrological Model Dataset